# 基础思路（baseline）

## 步骤1：更新或安装所需环境

In [1]:
!pip install --upgrade zhipuai requests urllib3 tqdm pandas
!apt update > /dev/null; apt install aria2 git-lfs axel -y > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.9.0
    Uninstalling PyJWT-2.9.0:
      Successfully uninstalled PyJWT-2.9.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, b

## 步骤2：下载数据集

In [2]:
!axel -n 12 -a https://ai-contest-static.xfyun.cn/2024/%E5%A4%A7%E6%A8%A1%E5%9E%8B%E8%83%BD%E5%8A%9B%E8%AF%84%E6%B5%8B%EF%BC%9A%E4%B8%AD%E6%96%87%E6%88%90%E8%AF%AD%E9%87%8A%E4%B9%89%E4%B8%8E%E8%A7%A3%E6%9E%90%E6%8C%91%E6%88%98%E8%B5%9B/test_input.csv

Initializing download: https://ai-contest-static.xfyun.cn/2024/%E5%A4%A7%E6%A8%A1%E5%9E%8B%E8%83%BD%E5%8A%9B%E8%AF%84%E6%B5%8B%EF%BC%9A%E4%B8%AD%E6%96%87%E6%88%90%E8%AF%AD%E9%87%8A%E4%B9%89%E4%B8%8E%E8%A7%A3%E6%9E%90%E6%8C%91%E6%88%98%E8%B5%9B/test_input.csv
File size: 131.332 Kilobyte(s) (134484 bytes)
Opening output file test_input.csv
Starting download

Can't setup alternate output. Deactivating.
...... .......... .......... .......... ..........  [  85.2KB/s]
[ 38%]  .......... .......... .......... .......... ..........  [ 167.0KB/s]
[ 76%]  .......... .......... .......... .

Downloaded 131.332 Kilobyte(s) in 0 second(s). (212.63 KB/s)


## 步骤3：构建模型（使用API调用glm-4-flash）

In [6]:
#同步调用
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=" ")  # 请填写您自己的APIKey 去智谱官网开个账号，充个20块钱就OK了，或者用免费的模型
response = client.chat.completions.create(
    model="glm-4-flash",  # 请填写您要调用的模型名称
    messages=[
        {"role": "user", "content": "你是一个中文文学大师，你的任务是用成语来描述中文语句。"},
        {"role": "assistant", "content": "列举与下面句子最相关的五个成语。只需要输出五个成语，不需要有其他的输出，写在一行中：双方心往一处想，意志坚定。"},
        {"role": "user", "content": "你是一个中文文学大师，你的任务是用成语来描述中文语句。"},
        {"role": "assistant", "content": "列举与下面句子最相关的五个成语。只需要输出五个成语，不需要有其他的输出，写在一行中：双方心往一处想，意志坚定。"},
        {"role": "user", "content": "列举与下面句子最相关的五个成语。只需要输出五个成语，不需要有其他的输出，写在一行中：双方心往一处想，意志坚定。"}
    ],
)
print("总消耗Tokens:",response.usage.total_tokens)
response = response.choices[0].message.content

print(response)

总消耗Tokens: 163
齐心协力、同心同德、众志成城、同心合意、志同道合


## 步骤4：读取数据集

In [4]:
import pandas as pd
test = pd.read_csv('./test_input.csv', header=None)

In [5]:
# 查看数据集大小
print(f"数据集的大小为: {test.shape[0]}\n前50条数据如下：\n")

# 查看前50条赛事数据集（赛题要求根据每行句子，给出5个可能匹配的成语）
for test_prompt in test[0].values[:50]:
    print(test_prompt)

数据集的大小为: 2972
前50条数据如下：

双方心往一处想，意志坚定。
两端都极为艰难，难以作出决定。
雄浑深远的意旨。细腻微妙。高超巧妙。
描述水流湍急，且快速且深远。
避免被引诱做出不道德或可疑的行为。
肘部肩窝。比喻事物发生于身边。
他张开嘴巴，吞咽着口水。
仅见一面，不足以深入了解。
向四处散发施舍，同时手中拿着碗盆。
比喻把装备脱下，放下武器。
对于任何战役，都要一败涂地。
阴谋真相已经昭然于众。
比喻因为无能为力而写下了好文章。
古代文人士大夫经常举行诗歌朗诵会。
不能形容为不高兴，只能说明没劲儿。
表现得举止端庄，很有教养。
1.洗刷兵器2.喂养战马3.准备作战
关注生命垂危者，关怀濒危者。
比喻面对挑战，坚韧不拔地前行。
过去的科举考试中被选拔为进士的称号。
相似程度极高或相差无几。
比喻不断地补充、堆砌和延伸。
以安逸快乐的生活和劳动为重。
她仍然每天教导她的儿子。
犹以火为耕，比喻原始、简朴的农耕方式。
指困境中处于不利地位。
搜集和研究其内在道理。
没有任何人帮助和支持。
这位作者的文章风格与自己非常相似。
国力强大，军事力量已经停止。
相互勾结维持；相互利用。
比喻进程飞快，日行千里。
只有一个目的，追求利润。
务必谨慎对待，慎重处理事务。
无法言表，只能感慨万千。
形容气势磅礴的文章风格。
根据贡献的大小给予奖励。
让国家蒙羞，民众蒙难。
形容有权势的人极其残忍和无礼。
坚决要求再次向某人强调。
采取措施；采取办法；采取行动；实行
心中充满了疑惑，还没有找到解答。
累累罪行，遍历无穷。形容罪恶极重。
形容人的容貌清爽俊雅，风度翩翩。
辞掉本职工作去做其他的事情。
秦汉时期，勋位至高者都佩戴金印和紫绶。
创立独特的风格，与众不同。
到处都是冰雪覆盖的环境，形容严冬天气。
旧事物被废弃；为了新事物而采取措施。
①向人行礼。②用作哀悼词或祭奠语。


## 步骤5：输出成语

In [8]:
from tqdm import tqdm
import os


i = 1
# 假设 test 是一个 DataFrame
# 遍历测试数据集的第一项的值，目的是生成与给定句子最相关的五个成语
for test_prompt in tqdm(test[0].values, total=len(test[0].values), desc="处理进度"):
    i = i + 1
    # 构造提示信息，要求模型输出与句子最相关的五个成语
    prompt = f"列举与下面句子最相关的五个成语。只需要输出五个成语，不需要有其他的输出，写在一行中：{test_prompt}"

    # 初始化一个长度为5的列表，填充默认成语“同舟共济”
    words = ['同舟共济'] * 5

    response = client.chat.completions.create(
    model="glm-4-flash",  # 请填写您要调用的模型名称
    messages=[
        {"role": "user", "content": "你是一个中文文学大师，你的任务是用成语来描述中文语句。"},
        {"role": "assistant", "content": prompt},
        {"role": "user", "content": "你是一个中文文学大师，你的任务是用成语来描述中文语句。"},
        {"role": "assistant", "content": prompt},
        {"role": "user", "content": prompt}
         ],
    )
    #response返回值中含有大量内容，有兴趣可以在此举前print一下。response中的content内容里包含完整的成语，进行提取
    response = response.choices[0].message.content

    # 清理回答文本，确保格式统一 GLM4生成结果中标点符号类型特别多增加了几个类型的标点符号去除。解决竖行输出的办法是去除'\n'换行符，使输出回到同一行内
    response = response.replace('\n', ' ').replace('、', ' ').replace(',', ' ').replace('；',' ').replace('。',' ')
    # 提取回答中的成语，确保每个成语长度为4且非空 这一句是导致全空多的原因
    words = [x for x in response.split() if len(x) == 4 and x.strip() != '']

    # 如果生成的成语列表长度不满足要求（不足20字符），则使用默认成语进行填充
    while True:
        text = ' '.join(words).strip()
        if len(text) < 24:
            words.append('同舟共济')
        else:
            break

    # 将最终的成语列表写入提交文件
    with open('submit.csv', 'a+', encoding='utf-8') as up:
        up.write(' '.join(words) + '\n')

    # 查看阶段性结果
    if i % 50 == 0:
        tqdm.write(f"大模型第{i}次返回的结果是：\n   {response}\n")
        tqdm.write(f"submit.cvs第{i}行输出结果：\n   {words}\n")

    # 为了尽快拿到结果，我们暂时仅获得500个结果（如果有时间的话，可以删除这两行）
    if i == 2973:
        break


print('submit.csv 已生成')

处理进度:   2%|▏         | 49/2972 [05:47<3:45:39,  4.63s/it]

大模型第50次返回的结果是：
   破旧立新 除旧布新 弃旧图新 革故鼎新 扬弃创新

submit.cvs第50行输出结果：
   ['破旧立新', '除旧布新', '弃旧图新', '革故鼎新', '扬弃创新']



处理进度:   3%|▎         | 99/2972 [06:53<1:07:35,  1.41s/it]

大模型第100次返回的结果是：
   知恩图报 知难而退 告辞而去 另辟蹊径 拂袖而去

submit.cvs第100行输出结果：
   ['知恩图报', '知难而退', '告辞而去', '另辟蹊径', '拂袖而去']



处理进度:   5%|▌         | 149/2972 [08:01<55:35,  1.18s/it]  

...

大模型第2900次返回的结果是：
   奖掖后学 扶植培养 纲举目张 抑扬顿挫 因势利导

submit.cvs第2900行输出结果：
   ['奖掖后学', '扶植培养', '纲举目张', '抑扬顿挫', '因势利导']



处理进度:  99%|█████████▉| 2949/2972 [1:14:51<00:26,  1.16s/it]

大模型第2950次返回的结果是：
   言辞谨慎 措辞得体 点到为止 含蓄委婉 言简意赅

submit.cvs第2950行输出结果：
   ['言辞谨慎', '措辞得体', '点到为止', '含蓄委婉', '言简意赅']



处理进度: 100%|█████████▉| 2971/2972 [1:15:24<00:01,  1.52s/it]

submit.csv 已生成
